In [2]:
import pysftp
import pandas as pd
import numpy as np
from pathlib import Path

In [2]:
#FTP Module
# cnopts = pysftp.CnOpts()
# cnopts.hostkeys = None
# sftp = pysftp.Connection('sftp.talentreef.com', username=hidden, password=hidden, cnopts=cnopts)
# sftp.get('/outbound/pde/TCF_18703/TCF_Terminations/TCF_Terminations20210802-1300-23.csv')
# print("Process Complete")

In [28]:
#Loading Employee Data
employee_data = pd.read_csv(Path("../Resources/Source_data/master.csv"), header = 0, usecols = ["EMP_NO","PAYCODE","PAYRATE"])
hour_data = pd.read_csv(Path("../Resources/Source_data/hours.csv")) 
employee_data   

,EMP_NO,PAYCODE,PAYRATE
0,101000,Salary,120000
1,101001,Hourly,15
2,101002,Salary,75000
3,101003,Hourly,40
4,101004,Salary,75000
5,101005,Hourly,45
6,101006,Hourly,17
7,101007,Hourly,15
8,101008,Salary,95000
9,101009,Hourly,15


In [42]:
#Calculating Hourly Wages
hourly_emp_data = employee_data.loc[employee_data["PAYCODE"] == "Hourly"]
hourly_grid = pd.merge(hourly_emp_data, hour_data, how = "inner", on = "EMP_NO")
hourly_grid["WAGES"] = np.where(hourly_grid["HOURS"] > 40, 
    ((hourly_grid["HOURS"]-40)*1.5*hourly_grid["PAYRATE"])+ (hourly_grid["PAYRATE"]*40), 
    (hourly_grid["HOURS"] * hourly_grid["PAYRATE"]))
hourly_grid.drop(["PAYCODE","PAYRATE","HOURS"], axis=1).round(2)

,EMP_NO,WAGES
0,101001,847.5
1,101003,1400.0
2,101005,1800.0
3,101006,1062.5
4,101007,825.0
5,101009,600.0
6,101010,680.0
7,101011,1400.0


In [43]:
#Calculating Salary Wages
salary_grid = employee_data.loc[employee_data["PAYCODE"] == "Salary"]
salary_grid["WAGES"] = salary_emp_data["PAYRATE"] / 52
salary_grid.drop(["PAYCODE","PAYRATE"], axis=1).round(2)


<ipython-input-43-ae01c44b3a5c>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  salary_grid["WAGES"] = salary_emp_data["PAYRATE"] / 52


,EMP_NO,WAGES
0,101000,2307.69
2,101002,1442.31
4,101004,1442.31
8,101008,1826.92


In [52]:
all_wages = hourly_grid.append(salary_grid, ignore_index=True, verify_integrity= True)
all_wages.drop(["PAYRATE","HOURS"], axis=1).round(2)

,EMP_NO,PAYCODE,WAGES
0,101001,Hourly,847.50
1,101003,Hourly,1400.00
2,101005,Hourly,1800.00
3,101006,Hourly,1062.50
4,101007,Hourly,825.00
5,101009,Hourly,600.00
6,101010,Hourly,680.00
7,101011,Hourly,1400.00
8,101000,Salary,2307.69
9,101002,Salary,1442.31


In [56]:
#Creating ACH File
employee_ach_data = pd.read_csv(Path("../Resources/Source_data/master.csv"), header = 0, usecols = ["EMP_NO","FIRST_NAME","BANK_RTN","BANK_ACC"])
ach_data = pd.merge(all_wages, employee_ach_data, how="inner", on = "EMP_NO")
ach_data.drop(["PAYCODE", "PAYRATE", "HOURS"], axis=1)

,EMP_NO,WAGES,FIRST_NAME,BANK_RTN,BANK_ACC
0,101001,847.500000,Jamel,124085244,1321541313
1,101003,1400.000000,Ivonne,91000019,21546931354
2,101005,1800.000000,Thomas,73905912,2136564123
3,101006,1062.500000,Marcell,124085244,16848643139846
4,101007,825.000000,Valarie,41215663,164864318771
5,101009,600.000000,Dewayne,124085244,569746414
6,101010,680.000000,Diamond,91408734,16874136
7,101011,1400.000000,Malik,256074974,1235483141
8,101000,2307.692308,Eric,256074974,1235483141
9,101002,1442.307692,Chudier,124085244,1215445
